In [3]:
import pandas as pd

# Cargar el DataFrame
df = pd.read_csv('../data/02_processed/df_final_modelar.csv')

# Contar el número de categorías únicas para 'Apartamento'
num_apartamentos = df['Apartamento'].nunique()
print(f'Número de categorías de Apartamento: {num_apartamentos}')

# Contar el número de categorías únicas para 'Portal de reserva'
num_portales = df['Portal de reserva'].nunique()
print(f'Número de categorías de Portal de reserva: {num_portales}')

Número de categorías de Apartamento: 41
Número de categorías de Portal de reserva: 4


In [4]:
# Contar el número de ocurrencias de cada categoría en 'Apartamento'
apartamento_counts = df['Apartamento'].value_counts()
print('\nNúmero de ocurrencias por categoría de Apartamento:')
print(apartamento_counts)

# Contar el número de ocurrencias de cada categoría en 'Portal de reserva'
portal_counts = df['Portal de reserva'].value_counts()
print('\nNúmero de ocurrencias por categoría de Portal de reserva:')
print(portal_counts)


Número de ocurrencias por categoría de Apartamento:
Apartamento
HD BRUNO                 691
H BMA PRAGA              685
H - BUA 3P               643
H BMA BERLIN             642
H - BUA 4P               624
H BMA OSLO               592
H BMA MONACO             560
HD ELENA                 560
HD CELESTE               538
H BMA AMSTERDAM          531
HD GLORIA                523
H BMA LISBOA             522
HD OLIVIA                516
HD FIDEL                 516
HD DARIO                 509
HD HUGHET                506
HD ALEJANDRA             506
H BMA HELSINKI           485
H BMA DUBLIN             479
H BMA CHICAGO            328
H BMA GARAJE             326
HG0 MUGARRA              323
HG1 ARTXANDA             321
HG1 ARRAIZ               317
HG0 PAGASARRI            309
HG0 COBETAS              296
HD-GARAJE 5              277
HG1 AMBOTO               269
HG0 GANETA               259
HG0 GANEKOGORTA          106
H - BUA GARAJE            76
AH10 INDIVIDUAL ARIAS     41
AH2 DOB

# CONSIDERACIONES ANTES de ENTRENAR MODELO ALGUNO

Diversidad en los Datos: La variedad de apartamentos y sus frecuencias indican una buena diversidad en tus datos, lo que puede ser útil para entrenar modelos de aprendizaje automático. Sin embargo, las categorías con pocas observaciones pueden afectar la robustez de tus modelos.
Posible Desbalance: Las categorías con muchas menos observaciones podrían ser minoritarias o atípicas, lo que puede necesitar técnicas de manejo de clases desbalanceadas si estás trabajando con modelos supervisados.

Las categorías de 'Portal de reserva' también muestran un gran rango en el número de ocurrencias, con 'Booking.com' siendo el más frecuente y 'Página web' el menos frecuente.
Desbalance en las Categorías: 'Booking.com' tiene una representación mucho mayor que las otras categorías, lo que puede indicar un sesgo en los datos o una preferencia muy marcada en el portal de reservas. Esto puede afectar el desempeño de los modelos de aprendizaje automático si no se maneja adecuadamente.

# Pasos a Seguir
**1. Manejo de Categorización y Desbalance**:

Agrupación de Categorías Menos Frecuentes: Para las categorías de apartamentos con pocas observaciones, podrías considerar agruparlas en una categoría "Otros" o realizar un análisis adicional para determinar si estas categorías tienen un impacto significativo en el análisis.
Estratificación y Balanceo: Para el portal de reservas, podrías considerar técnicas de balanceo como sobremuestreo (SMOTE) o submuestreo para manejar el desbalance en las categorías.

**2. Preparación para Clustering**:

Codificación de Variables Categóricas: Convierte las variables categóricas a una representación numérica adecuada. Técnicas como One-Hot Encoding o Codificación de Etiquetas podrían ser útiles dependiendo del algoritmo de clustering que elijas.
Normalización y Escalado: Asegúrate de que todas las variables numéricas estén normalizadas o escaladas para que los algoritmos de clustering, que son sensibles a las magnitudes, funcionen correctamente.

# K-Means

Vamos con todo sin volvernos locos, a ver que sale.

## 1.- IMPLEMENTAMOS

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']),
        ('cat', OneHotEncoder(), ['Apartamento', 'Portal de reserva','Estado'])
    ])

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clusterer', KMeans(n_clusters=5))  # Cambia el número de clusters según sea necesario
])

# Ajustar el modelo
pipeline.fit(df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Precio', 'Número de noches',
                                                   'ADR',
                                                   'Precio_Medio_Historico',
                                                   'Porcentaje_Ocupacion']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Apartamento',
                                                   'Portal de reserva',
                                                   'Estado'])])),
                ('clusterer', KMeans(n_clusters=5))])

## 2.- EVALUAMOS

### Método del codo: 
SSE (Suma de los Errores Cuadráticos Internos)

In [7]:

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']),
        ('cat', OneHotEncoder(), ['Apartamento', 'Portal de reserva'])
    ])

# Preprocesar los datos
X = preprocessor.fit_transform(df)

# Calcular el SSE para diferentes números de clusters
sse = []
k_values = range(1, 40)  # Prueba con k de 1 a 20

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

# Mostrar SSE para cada k
for k, score in zip(k_values, sse):
    print(f'Número de clusters: {k}, SSE: {score}')

Número de clusters: 1, SSE: 90618.99266055046
Número de clusters: 2, SSE: 75232.47332912654
Número de clusters: 3, SSE: 65770.00987817632
Número de clusters: 4, SSE: 57665.779064494694
Número de clusters: 5, SSE: 52596.25938552825
Número de clusters: 6, SSE: 48919.7965279566
Número de clusters: 7, SSE: 44576.12326329837
Número de clusters: 8, SSE: 42909.9431100869
Número de clusters: 9, SSE: 40794.96220713247
Número de clusters: 10, SSE: 39536.39496493123
Número de clusters: 11, SSE: 38309.32706784505
Número de clusters: 12, SSE: 36863.8982832744
Número de clusters: 13, SSE: 35554.27082197393
Número de clusters: 14, SSE: 34531.67023874862
Número de clusters: 15, SSE: 33664.097725047184
Número de clusters: 16, SSE: 33184.845269899255
Número de clusters: 17, SSE: 32536.469985134605
Número de clusters: 18, SSE: 31983.608763094187
Número de clusters: 19, SSE: 29996.544974614353
Número de clusters: 20, SSE: 29292.334665874838
Número de clusters: 21, SSE: 29045.77625793109
Número de clusters

### Método Silhouette

KMEANS - 2 cluster

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd

# Supongamos que tienes el DataFrame df

# Definir las características para el clustering
features = ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion', 'Apartamento', 'Portal de reserva']

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']),
        ('cat', OneHotEncoder(), ['Apartamento', 'Portal de reserva'])
    ])

# Crear el pipeline para el clustering
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clusterer', KMeans(n_clusters=2))  # Puedes ajustar el número de clusters
])

# Seleccionar solo las columnas relevantes
X = df[features]

# Ajustar el modelo
pipeline.fit(X)

# Obtener las etiquetas de cluster
df['Cluster'] = pipeline.predict(X)

In [10]:
# Evaluar el modelo usando Silhouette Score
clustered_data = pipeline.transform(X)  # Obtener las características transformadas
silhouette_avg = silhouette_score(clustered_data, df['Cluster'])
print(f'Silhouette Score para n_clusters=2: {silhouette_avg}')


Silhouette Score para n_clusters=2: 0.42887782924726064


KMEANS - 5 cluster

In [11]:
# Definir las características para el clustering
features = ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion', 'Apartamento', 'Portal de reserva']

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']),
        ('cat', OneHotEncoder(), ['Apartamento', 'Portal de reserva'])
    ])

# Crear el pipeline para el clustering
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clusterer', KMeans(n_clusters=5))  # Puedes ajustar el número de clusters
])

# Seleccionar solo las columnas relevantes
X = df[features]

# Ajustar el modelo
pipeline.fit(X)

# Obtener las etiquetas de cluster
df['Cluster'] = pipeline.predict(X)

In [12]:
# Evaluar el modelo usando Silhouette Score
clustered_data = pipeline.transform(X)  # Obtener las características transformadas
silhouette_avg = silhouette_score(clustered_data, df['Cluster'])
print(f'Silhouette Score para n_clusters=5: {silhouette_avg}')


Silhouette Score para n_clusters=5: 0.20818649724244473


KMEANS - 7 cluster

In [13]:
# Definir las características para el clustering
features = ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion', 'Apartamento', 'Portal de reserva']

# Definir el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']),
        ('cat', OneHotEncoder(), ['Apartamento', 'Portal de reserva'])
    ])

# Crear el pipeline para el clustering
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clusterer', KMeans(n_clusters=7))  # Puedes ajustar el número de clusters
])

# Seleccionar solo las columnas relevantes
X = df[features]

# Ajustar el modelo
pipeline.fit(X)

# Obtener las etiquetas de cluster
df['Cluster'] = pipeline.predict(X)

In [14]:
# Evaluar el modelo usando Silhouette Score
clustered_data = pipeline.transform(X)  # Obtener las características transformadas
silhouette_avg = silhouette_score(clustered_data, df['Cluster'])
print(f'Silhouette Score para n_clusters=7: {silhouette_avg}')


Silhouette Score para n_clusters=7: 0.27527290273263755


**BEST Silhouette Score n_clusters=2 : 0.42893918485879795**

**BEST n_cluster SSE = 7**

Existe una discrepancia clara entre el método del Codo (SSE) y el Índice de Silhouette. 

.-.-.-.-.-
# ANÁLISIS de COMPONENTES

In [16]:
from sklearn.decomposition import PCA

# Ajustar el preprocesador (solo) para obtener los datos transformados
X_transformed = pipeline.named_steps['preprocessor'].transform(df)

# Aplicar PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_transformed)

# Aplicar PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_transformed)

# Obtener los nombres de las características después del OneHotEncoding
num_features = pipeline.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out()
cat_features = pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out()
feature_names = list(num_features) + list(cat_features)

# Obtener las cargas de las características
pca_loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=feature_names)
print(pca_loadings)

                                        PC1       PC2
Precio                             0.617194  0.301928
Número de noches                   0.401517  0.633608
ADR                                0.516780 -0.345265
Precio_Medio_Historico             0.407677 -0.509044
Porcentaje_Ocupacion               0.131390 -0.330745
Apartamento_AH1 DOBLE ARIAS       -0.001362  0.000618
Apartamento_AH10 INDIVIDUAL ARIAS -0.001784  0.001155
Apartamento_AH11 DOBLE ARIAS      -0.001170  0.000281
Apartamento_AH12 DOBLE ARIAS      -0.000941  0.000470
Apartamento_AH13 INDIVIDUAL ARIAS -0.001605  0.000823
Apartamento_AH14 INDIVIDUAL ARIAS -0.001213  0.000626
Apartamento_AH2 DOBLE ARIAS       -0.000831  0.000328
Apartamento_AH4 INDIVIDUAL ARIAS  -0.000922  0.000770
Apartamento_AH6 INDIVIDUAL ARIAS  -0.001466  0.000896
Apartamento_AH9 DOBLE ARIAS       -0.001364  0.000430
Apartamento_H - BUA 3P            -0.016550  0.015763
Apartamento_H - BUA 4P            -0.012904  0.011216
Apartamento_H - BUA GARAJE  

# PC1:

## Precio (0.617194): 
### Alta carga positiva, lo que indica que el precio tiene una fuerte correlación positiva con PC1. Un aumento en el precio tiende a aumentar el valor de PC1.
## Número de noches (0.401517): 
### Carga positiva moderada, lo que sugiere que un mayor número de noches también está positivamente correlacionado con PC1, pero no tanto como el precio.
## ADR (0.516780): 
### Carga positiva, indicando que un mayor ADR está asociado con un mayor valor en PC1.
## Precio_Medio_Historico (0.407677): 
### También carga positivamente, indicando una relación similar con el precio y ADR en PC1.
## Porcentaje_Ocupacion (0.131390): 
### Carga baja positiva, indicando una correlación débil con PC1.

# PC2:

## Número de noches (0.633608): 
### Carga positiva alta, lo que indica que el número de noches tiene una fuerte influencia en PC2. Los datos con más noches tienden a tener valores más altos en PC2.
## Precio (0.301928): 
### Carga positiva baja, por lo que el precio tiene una influencia menor en PC2.
## ADR (-0.345265): 
### Carga negativa moderada, lo que sugiere que un aumento en ADR tiende a reducir el valor de PC2.
## Precio_Medio_Historico (-0.509044): 
### Carga negativa significativa, indicando que un mayor precio medio histórico está asociado con valores más bajos en PC2.
## Porcentaje_Ocupacion (-0.330745): 
### Carga negativa, indicando una relación inversa con PC2. A medida que el porcentaje de ocupación aumenta, el valor de PC2 tiende a disminuir.

# Selección del Número Óptimo de Componentes

In [19]:
import numpy as np

# Definir las columnas categóricas y numéricas
categorical_features = ['Apartamento', 'Portal de reserva']
numeric_features = ['Precio', 'Número de noches', 'ADR', 'Precio_Medio_Historico', 'Porcentaje_Ocupacion']

# Crear los transformadores para las columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Aplicar el preprocesador a los datos
X = preprocessor.fit_transform(df)

# Aplicar PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# Obtener la varianza explicada
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)
print(f'Varianza explicada acumulativa: {cumulative_variance}')

# Decidir cuántos componentes usar, por ejemplo, para capturar al menos el 80% de la varianza
n_components = np.argmax(cumulative_variance >= 0.80) + 1

# Redimensionar los datos usando PCA con el número de componentes seleccionados
pca = PCA(n_components=n_components)
X_pca_reduced = pca.fit_transform(X)

# Aplicar KMeans
kmeans = KMeans(n_clusters=2)
clusters = kmeans.fit_predict(X_pca_reduced)

# Evaluar el clustering
score = silhouette_score(X_pca_reduced, clusters)
print(f'Silhouette Score: {score:.2f}')


Varianza explicada acumulativa: [0.33573357 0.54838925 0.69556328 0.77672708 0.82190701 0.83830056
 0.85231002 0.85989687 0.8672971  0.87428434 0.88117413 0.88777556
 0.89411383 0.90027526 0.90629463 0.91218975 0.91795413 0.92364902
 0.92931271 0.93491886 0.94050279 0.94593239 0.95125382 0.95624455
 0.96077235 0.96492807 0.96855771 0.97210046 0.9756143  0.97904527
 0.98236801 0.98566382 0.98871327 0.99158871 0.9936918  0.99489669
 0.99601171 0.99658922 0.9970253  0.99742286 0.99781985 0.99821635
 0.99859766 0.998973   0.9993198  0.99964551 0.99989844 1.
 1.        ]
Silhouette Score: 0.31


In [20]:
# Umbrales de varianza explicada
thresholds = [0.80, 0.90]

# Encontrar el número de componentes necesarios para cada umbral
n_components_80 = np.argmax(cumulative_variance >= thresholds[0]) + 1
n_components_90 = np.argmax(cumulative_variance >= thresholds[1]) + 1

print(f'Número de componentes para capturar al menos el 80% de la varianza: {n_components_80}')
print(f'Número de componentes para capturar al menos el 90% de la varianza: {n_components_90}')


Número de componentes para capturar al menos el 80% de la varianza: 5
Número de componentes para capturar al menos el 90% de la varianza: 14
